In [ ]:
import requests 
import json
import pandas as pd
import numpy as np
from numpy import array
from datetime import datetime, timedelta

In [ ]:
# Creating a list of days in unix time to make dark sky api calls with
def list_of_days():
    days_list = []
    starting_x = 1546214400
    while starting_x < 1572480000:
        starting_x = starting_x + 86400
        
        days_list.append(starting_x)
    return days_list

In [ ]:
ut = list_of_days()

In [ ]:
ut

In [ ]:
# creating a list of links with a loop that merges our base with the unix date time info
def create_link():
    base = 'https://api.darksky.net/forecast/{API Key}/40.6782,-73.9442,' 
    end = '?exclude=currently,minutely,hourly,alerts,flags'
    list_of_links = []
    for i in ut:
        list_of_links.append(base + str(i) + end)
    return list_of_links

In [ ]:
links_list = create_link()

In [ ]:
len(links_list)

In [ ]:
# creating a function to make API calls with our list of url links
def create_weather():
    weather_data = [] 
    for i in links_list:
        x = requests.get(i)
        weather_data.append(x.content)
    return weather_data

In [ ]:
weather_data_list = create_weather()

In [ ]:
weather_data = weather_data_list

In [ ]:
weather_data

In [ ]:
#removing a layer of the dictionaries received from our API calls
new_list = []
for i in weather_data_list:
    p = json.loads(i)
    x = {}
    for k,v in p.items():
        if k == 'daily':
            x['daily'] = v
    new_list.append(x)
new_list

In [ ]:
len(new_list)

In [ ]:
#removing another layer of the dictionaries received from our API calls
cleaner_weather = []
for i in new_list:
    for k,v in i.items():
        x = v
        cleaner_weather.append(x)
cleaner_weather

In [ ]:
len(cleaner_weather)

In [ ]:
#removing a layer of the dictionaries received from our API calls to create our final weather data set
cleaner_weather1 = []
for i in cleaner_weather:
    for k,v in i.items():
        x = v
        cleaner_weather1.append(x)
cleaner_weather1

In [ ]:
len(cleaner_weather1)

In [ ]:
#function to create pull only the relevant info from our API calls
cw = []
for i in cleaner_weather1:
    for x in i: 
        final_weather_list = {}
        for k,v in x.items():
            if k == 'time':
                final_weather_list['time'] = v
            if k == 'icon':
                final_weather_list['icon'] = v
            if k == 'moonPhase':
                final_weather_list['moonPhase'] = v
            if k == 'precipIntensity':
                final_weather_list['precipIntensity'] = v
            if k == 'apparentTemperatureHigh':
                final_weather_list['apparentTemperatureHigh'] = v
            if k == 'apparentTemperatureLow':
                final_weather_list['apparentTemperatureLow'] = v
            if k == 'windSpeed':
                final_weather_list['windSpeed'] = v
        cw.append(final_weather_list)
        
dfcw = pd.DataFrame(cw)
dfcw

In [ ]:
halloween = requests.get('https://api.darksky.net/forecast/7094f90a2c5f593cd44c74c5192618dd/40.6782,-73.9442,1572494400?exclude=currently,minutely,hourly,alerts,flags')

In [ ]:
halloween.content

In [ ]:
#converting unix time to datetime so we can merge our datasets together later
dfcw['time'] = pd.to_datetime(dfcw['time'], unit='s')

In [ ]:
dfcw

In [ ]:
dfcw.drop([0], inplace=True)

In [ ]:
dfcw

In [ ]:
dfcw.columns

In [ ]:
#creating a new column in our dataframe with the mean of our high and low temps
dfcw['Average_Temp'] = dfcw[['apparentTemperatureHigh', 'apparentTemperatureLow']].apply(lambda x: (x['apparentTemperatureHigh'] + x['apparentTemperatureLow'])/2, axis = 1)

In [ ]:
dfcw

In [ ]:
#dropping temp columns
dfcw.drop(['apparentTemperatureHigh', 'apparentTemperatureLow'], axis = 1, inplace=True)

In [ ]:
dfcw.count()

In [ ]:
dfcw = dfcw.dropna(how='any', axis=0)

In [ ]:
dfcw.count()

In [ ]:
dfcw.head()

In [ ]:
dfcw['time'] = pd.to_datetime(dfcw['time'])

In [ ]:
dfcw.info()

In [ ]:
dfcw['time'] = pd.to_datetime(dfcw['time']).dt.normalize()

In [ ]:
dfcw

In [ ]:
dfcw.rename(columns={'time': 'Date', 'icon': "Forecast", 'moonPhase': 'Moon Phase', 'precipIntensity': 'Precipitation', 'windSpeed': 'Wind Speed'}, inplace=True)

In [ ]:
dfcw